In [2]:
%%configure -f
{
    "driverMemory": "8g",
    "driverCores": 4,
    "executorMemory": "8g",
    "executorCores": 4,
    "numExecutors": 2,
    "conf":{
        "spark.sql.shuffle.partitions": 200,
        "spark.dynamicAllocation.enabled": "true",
        "spark.dynamicAllocation.minExecutors": 2,
        "spark.dynamicAllocation.maxExecutors": 8,
        "spark.sql.sources.useV1SourceList":"true",
        "spark.sql.join.preferSortMergeJoin":"false",
        "spark.shuffle.manager": "org.apache.spark.shuffle.sort.ColumnarShuffleManager",
        "spark.gluten.ras.enabled": "true",
        "spark.gluten.sql.columnar.backend.velox.bloomFilter.expectedNumItems": 1000000,
        "spark.gluten.sql.columnar.backend.velox.bloomFilter.numBits": 8388608,
        "spark.gluten.sql.columnar.backend.velox.bloomFilter.maxNumBits": 4194304,
        "spark.gluten.velox.awsSdkLogLevel": "FATAL"
    }
} 


StatementMeta(SmallPool, 32, -1, Finished, Available, Finished)

See https://go.microsoft.com/fwlink/?linkid=2170827


Synapse runtime contains Apache Gluten/Velox, a C++ SQL execution engine.

This notebook shows how to enable, allocate memory and check Apache Gluten/Velox.

The above configuration is for "Small" Spark pool.

The following 3 configuration items need to be placed a configuration in Manage --> Apache Spark configurations, and linked to the pool or notebook
- "spark.plugins":"org.apache.gluten.GlutenPlugin"
- "spark.gluten.enabled": "true"
- "spark.memory.offHeap.size": "20g"

The last configuration "spark.memory.offHeap.size" assigns 20GB of a small node's 28GB to Gluten. Gluten memory is not managed by JVM, thus "offHeap". Please adapt memory size for node size.

From [Gluten](https://github.com/apache/incubator-gluten/blob/main/docs/Configuration.md)

Available memory size of Synapse Spark nodes: 
- Small: 28g
- Medium: 56g
- Large: 112g
- XLarge: 224g

Note: We **cannot** configure spark.executor.memory in Spark configuration. It will have conflict with Synapse's configuration of the same items.

From Microsoft: [Magic command for configuring a Spark session](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-development-using-notebooks#spark-session-config-magic-command)
```
%%configure
{
    //You can get a list of valid parameters to configure the session from https://github.com/cloudera/livy#request-body.
    "driverMemory":"28g", // Recommended values: ["28g", "56g", "112g", "224g", "400g", "472g"]
    "driverCores":4, // Recommended values: [4, 8, 16, 32, 64, 80]
    "executorMemory":"28g",
    "executorCores":4, 
    "jars":["abfs[s]://<file_system>@<account_name>.dfs.core.windows.net/<path>/myjar.jar","wasb[s]://<containername>@<accountname>.blob.core.windows.net/<path>/myjar1.jar"],
    "conf":{
    //Example of a standard Spark property. To find more available properties, go to https://spark.apache.org/docs/latest/configuration.html#application-properties.
        "spark.driver.maxResultSize":"10g",
    //Example of a customized property. You can specify the count of lines that Spark SQL returns by configuring "livy.rsc.sql.num-rows".
        "livy.rsc.sql.num-rows":"3000" 
    }
}
```


#### Check memory configuration

In [5]:
spark.conf.get('spark.executor.memory'),spark.conf.get('spark.memory.offHeap.size')

StatementMeta(SmallPool, 32, 4, Finished, Available, Finished)

('8g', '20g')

#### Check if Gluten/Velox is enabled

We can test if Gluten/Velox is in use by EXPLAIN querying a parquet over 5MB. If the plan contains 'Velox', it means Gluten/Velox is in effect

In [6]:
print(spark.sql('EXPLAIN SELECT * FROM parquet.`abfss://******@******.dfs.core.windows.net/disease.parquet`').collect()[0].plan)

# example parquet file thanks to https://platform-docs.opentargets.org/data-access/datasets. Put it in your ADLFS G2 container before execution

StatementMeta(SmallPool, 32, 5, Finished, Available, Finished)

== Physical Plan ==
VeloxColumnarToRow
+- ^(2) FileScanTransformer parquet [id#25,code#26,name#27,description#28,dbXRefs#29,parents#30,synonyms#31,obsoleteTerms#32,obsoleteXRefs#33,children#34,ancestors#35,therapeuticAreas#36,descendants#37,ontology#38] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[abfss://******@*******.dfs.core.windows.net/disease.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:string,code:string,name:string,description:string,dbXRefs:array<string>,parents:array<s... NativeFilters: []


